<a href="https://colab.research.google.com/github/ROYALKINGISK/ML-LAB/blob/main/ML_BL_EN_U4CSE21070_LAB8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#A1
import pandas as pd

# Replace this with your actual dataset
data = {
    'age': ['<=30','<=30','31...40','>40','>40','>40','31...40','<=30','<=30','>40','<=30','31...40','31...40','>40'],
    'income': ['high', 'high','high','medium','low','low','low','medium','low','mediun','medium','medium','high','medium'],
    'student': ['no', 'no', 'no', 'no','yes', 'yes', 'yes','no', 'yes','yes','yes', 'no', 'yes', 'no'],
    'credit_rating': ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair','fair','excellent', 'excellent','fair','excellent'],
    'buys_computer': ['no', 'no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
}

df = pd.DataFrame(data)

# Count the occurrences of each class
class_counts = df['buys_computer'].value_counts()

# Calculate prior probability for each class
prior_probabilities = class_counts / len(df)

# Display the results
print("Class Counts:")
print(class_counts)
print("\nPrior Probabilities:")
print(prior_probabilities)


Class Counts:
yes    9
no     5
Name: buys_computer, dtype: int64

Prior Probabilities:
yes    0.642857
no     0.357143
Name: buys_computer, dtype: float64


In [20]:
#A2
from sklearn.neighbors import KernelDensity
import numpy as np
age_mapping = {'<=30': 1, '31...40': 2, '>40': 3}
df['age_numeric'] = df['age'].map(age_mapping)

# Map income categories to numerical values
income_mapping = {'low': 1, 'medium': 2, 'high': 3}
df['income_numeric'] = df['income'].map(income_mapping)

# Iterate through features and calculate class conditional densities
for feature in ['age_numeric', 'income_numeric', 'student', 'credit_rating']:
    for class_label in df['buys_computer'].unique():
        # Extract data for the specific class
        subset = df[df['buys_computer'] == class_label][feature].values.reshape(-1, 1)

        # Use kernel density estimation
        kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
        kde.fit(subset)

        # Generate some sample data points
        sample_points = np.linspace(df[feature].min(), df[feature].max(), 100).reshape(-1, 1)

        # Calculate the log density for the sample points
        log_density = kde.score_samples(sample_points)

        # Display the results
        print(f"Class: {class_label}, Feature: {feature}")
        print("Sample Points:", sample_points.flatten())
        print("Log Density:", log_density)
        print()

Class: no, Feature: age_numeric
Sample Points: [1.         1.02020202 1.04040404 1.06060606 1.08080808 1.1010101
 1.12121212 1.14141414 1.16161616 1.18181818 1.2020202  1.22222222
 1.24242424 1.26262626 1.28282828 1.3030303  1.32323232 1.34343434
 1.36363636 1.38383838 1.4040404  1.42424242 1.44444444 1.46464646
 1.48484848 1.50505051 1.52525253 1.54545455 1.56565657 1.58585859
 1.60606061 1.62626263 1.64646465 1.66666667 1.68686869 1.70707071
 1.72727273 1.74747475 1.76767677 1.78787879 1.80808081 1.82828283
 1.84848485 1.86868687 1.88888889 1.90909091 1.92929293 1.94949495
 1.96969697 1.98989899 2.01010101 2.03030303 2.05050505 2.07070707
 2.09090909 2.11111111 2.13131313 2.15151515 2.17171717 2.19191919
 2.21212121 2.23232323 2.25252525 2.27272727 2.29292929 2.31313131
 2.33333333 2.35353535 2.37373737 2.39393939 2.41414141 2.43434343
 2.45454545 2.47474747 2.49494949 2.51515152 2.53535354 2.55555556
 2.57575758 2.5959596  2.61616162 2.63636364 2.65656566 2.67676768
 2.6969697  2.71

ValueError: ignored

In [10]:
#A3
from scipy.stats import chi2_contingency, pearsonr
categorical_features = ['student', 'credit_rating', 'buys_computer']

for feature in categorical_features:
    contingency_table = pd.crosstab(df[feature], df['buys_computer'])
    chi2, p, _, _ = chi2_contingency(contingency_table)
    print(f"Chi-squared test for independence between {feature} and buys_computer:")
    print(f"Chi-squared statistic: {chi2}")
    print(f"P-value: {p}")
    print(f"{'*' * 40}")

# Numeric features: Pearson correlation coefficient
numeric_features = ['age_numeric', 'income_numeric']

for feature in numeric_features:
    # Drop rows with missing values
    valid_data = df.dropna(subset=[feature, 'age_numeric'])

    correlation, p_value = pearsonr(valid_data[feature], valid_data['age_numeric'])
    print(f"Pearson correlation coefficient between {feature} and age_numeric:")
    print(f"Correlation coefficient: {correlation}")
    print(f"P-value: {p_value}")
    print(f"{'*' * 40}")

Chi-squared test for independence between student and buys_computer:
Chi-squared statistic: 1.2444444444444445
P-value: 0.2646162170835855
****************************************
Chi-squared test for independence between credit_rating and buys_computer:
Chi-squared statistic: 0.16203703703703706
P-value: 0.6872879493480019
****************************************
Chi-squared test for independence between buys_computer and buys_computer:
Chi-squared statistic: 9.983209876543212
P-value: 0.0015797405840629584
****************************************
Pearson correlation coefficient between age_numeric and age_numeric:
Correlation coefficient: 0.9999999999999999
P-value: 0.0
****************************************
Pearson correlation coefficient between income_numeric and age_numeric:
Correlation coefficient: -0.35507405735549946
P-value: 0.23383709750448237
****************************************


In [13]:
#A4
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


# Separate features and target variable
X = df[['age', 'income', 'student', 'credit_rating']]
y = df['buys_computer']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical features to a dictionary format
train_features = X_train.to_dict(orient='records')
test_features = X_test.to_dict(orient='records')

# Vectorize features using DictVectorizer
vectorizer = DictVectorizer(sparse=False)
X_train_vec = vectorizer.fit_transform(train_features)
X_test_vec = vectorizer.transform(test_features)

# Build a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test_vec)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Display the results
print("Naïve Bayes Classifier Results:")
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_rep)


Naïve Bayes Classifier Results:
Accuracy: 0.33
Classification Report:
               precision    recall  f1-score   support

          no       0.00      0.00      0.00         1
         yes       0.50      0.50      0.50         2

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3



In [15]:
#A5
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Replace this with your actual dataset
data = pd.read_csv("extracted_features_charrec.csv")
df = pd.DataFrame(data)

# Separate features and target variable
X = df.drop(['class_name'], axis=1)
y = df['class_name']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical features to a dictionary format
train_features = X_train.to_dict(orient='records')
test_features = X_test.to_dict(orient='records')

# Vectorize features using DictVectorizer
vectorizer = DictVectorizer(sparse=False)
X_train_vec = vectorizer.fit_transform(train_features)
X_test_vec = vectorizer.transform(test_features)

# Build a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test_vec)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Display the results
print("Naïve Bayes Classifier Results:")
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_rep)


Naïve Bayes Classifier Results:
Accuracy: 0.65
Classification Report:
               precision    recall  f1-score   support

        3333       0.45      0.94      0.61        36
        3334       0.48      0.94      0.64        33
        3335       0.33      0.72      0.45        39
        3337       0.58      0.53      0.55        34
        3342       0.52      0.57      0.54        28
        3343       0.00      0.00      0.00        26
        3346       0.48      0.57      0.52        21
        3349       0.73      0.64      0.68        47
        3350       0.00      0.00      0.00        20
        3351       0.95      0.84      0.89        43
        3352       0.00      0.00      0.00        22
        3353       0.50      0.86      0.63        29
        3354       0.88      0.77      0.82        30
        3355       0.00      0.00      0.00        20
        3356       0.50      0.84      0.63        32
        3357       1.00      0.04      0.07        27
        33

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
